This tutorial is taken from https://github.com/KeithGalli/pandas/tree/master
The video to this tutorial is here: https://youtu.be/vmEHCJofslg


## Loading data into Pandas

In [ ]:
# prompt: download the file https://github.com/KeithGalli/pandas/blob/master/pokemon_data.csv

import requests

def download_file(url, filename):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes

        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"File '{filename}' downloaded successfully.")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")

# Example usage:
file_url = "https://raw.githubusercontent.com/KeithGalli/pandas/master/pokemon_data.csv" # Note: using raw content link
file_name = "pokemon_data.csv"

download_file(file_url, file_name)

In [ ]:
# prompt: open the downloaded file with pandas

import pandas as pd

# Assuming the file is named 'pokemon_data.csv' in the current directory
try:
  df = pd.read_csv('pokemon_data.csv', index_col='Name')
  print(df.head()) # Print the first few rows of the DataFrame
except FileNotFoundError:
  print("Error: 'pokemon_data.csv' not found. Please make sure the file exists in the current directory or provide the correct path.")
except pd.errors.ParserError:
    print("Error: Could not parse the CSV file. Please check the file format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

## Accessing data

Read column headers:

In [ ]:
df.columns

Get a column as the `pandas.core.series.Series` object:

In [ ]:
df['HP']

Get a DataFrame containing just a subset of columns:

In [ ]:
print(type(df[['Type 1', 'Type 2', 'HP']]))

Given that we've chosen `index_col='Name'` in `read_csv` above, we can use the pokemons' name as index. Note the usage of the `loc` method for selecting columns and rows. This method makes the selection based on predefined labels of the rows and columns. We will use this method a lot in this notebook!

In [ ]:
df.loc['Ivysaur']

The returned object is `pandas.Series`. We can access its elements using square brackets, e.g. `pokemon['Type 2']`:

In [ ]:
df.loc['Ivysaur']['Type 2']

The `iloc` method makes the columns and rows selection based on their position in the DataFrame. The position is numbered by an integer. The following code returns the third row in our DataFrame:

In [ ]:
df.iloc[2]

Slicing rows with `iloc`:

In [ ]:
print(df.iloc[0:4])

Read the "cell" at coordinates [2,1] (row, column):

In [ ]:
print(df.iloc[2,1])

Select a subset of rows (just those where the pokemon's `Type 1` is `Grass`)

In [ ]:
df.loc[df['Type 1'] == "Grass"]

Note that ```df['Type 1'] == "Grass"``` creates an array of bool values. So, the `loc` method accepts an array of bools as input. Therefore, we can also do things like the following with bool columns:

In [ ]:
df.loc[df['Legendary']]

Apply multiple selection criteria using the logical AND operator `&`. Note the parentheses around the `df['Type 1'] == "Fire"` condition. They must be there because the `&` operatore, bitwise AND, has lower precedence than the `==` equality operator.

In [ ]:
df.loc[df['Legendary'] & (df['Type 1'] == "Fire")]

Logical OR operation:

In [ ]:
df.loc[(df['Type 1'] == 'Grass') | (df['Type 1'] == "Fire")]

After the filtering, one can ask question like what is the highest `"Attack"` of a pokemon that passed.

In [ ]:
fire_legendary_pokemon = df.loc[(df['Type 1'] == 'Fire') & (df['Legendary'] == True)]

if not fire_legendary_pokemon.empty:
  pokemon_with_highest_attack = fire_legendary_pokemon.loc[fire_legendary_pokemon['Attack'].idxmax()]

The operator `~` is the NOT operator. The following expression selects all pokemons that are not Legendary and have the `"HP"` value greater than 150.

In [ ]:
df.loc[~df['Legendary'] & (df['HP'] > 150)]

To count the number of `"Type 1"` species, use the `value_counts` method.

In [ ]:
df['Type 1'].value_counts()

Select pokemons with Defense in the `(40, 60)` interval:

In [ ]:
new_df = df.loc[df['Defense'].between(40, 60)]
new_df.head()

When we want pokemons with some specific `"Defense"` values, we use the `isin` method.

In [ ]:
df.loc[df['Defense'].isin([40, 70, 30, 60])].head()

Select 5% pokemons with the highest speed. The method `quantile` is good for this! (Note that we will not select exactly 5% pokemons because there are several pokemons having the same speed commonly.)

In [ ]:
new_df = df.loc[df['Speed'] > df['Speed'].quantile(0.95)]
print(new_df.shape)
print(df.shape)
print(new_df.shape[0] / df.shape[0])

## Sorting/Describing Data

To sort the rows according to the `"HP"` value in descending order, do:

In [ ]:
df.sort_values('HP', ascending=False)

One can also sort according to several characteristics. The rows are sorted primarily according to the `"HP"` value in the descending order. If several pokemons have the same `"HP"` value then they are sorted according to the `"Attack"` value in ascending order.

In [ ]:
df.sort_values(['HP', 'Attack'], ascending=[True, False])

## Making changes to the data

Create a new column that contains a sum of values from 6 chosen columns:

In [ ]:
df['Total'] = df['HP'] + df['Attack'] + df['Defense'] + df['Sp. Atk'] + df['Sp. Def'] + df['Speed']
df['Total']

Remove the column

In [ ]:
df = df.drop(columns=['Total'])

Create it once again, calculating the same sum as previously:

In [ ]:
df['Total'] = df.iloc[:, 4:10].sum(axis=1)
df['Total']

Reorder columns:

In [ ]:
cols = list(df.columns)
df = df[cols[0:4] + [cols[-1]]+cols[4:11]]

df.head(5)

`reset_index` method updates indeces in a way you choose. `inplace=True` means that no new DataFrame is created. `drop=True` means that any previous index is removed from the DataFrame. With `drop=False`, the old index would be added as a new column to the DataFrame.

In [ ]:
new_df = df.loc[(df['Type 1'] == 'Grass') & (df['Type 2'] == 'Poison') & (df['HP'] > 70)]
new_df.reset_index(drop=True, inplace=True)

Conditional changes

In [ ]:
df.loc[df['Attack'] > 100, ['Generation','Legendary']] = [10, True]
df.loc[df['Attack'] > 100]

## Creating DataFrames

In [ ]:
# prompt: examples of new dataframe creation

# Method 1: From a dictionary
data = {'col1': [1, 2, 3], 'col2': [4, 5, 6]}
df_from_dict = pd.DataFrame(data)
print("DataFrame from dictionary:\n", df_from_dict)

# Method 2: From a list of lists
data = [[1, 4], [2, 5], [3, 6]]
columns = ['col1', 'col2']
df_from_list = pd.DataFrame(data, columns=columns)
print("\nDataFrame from list of lists:\n", df_from_list)

# Method 3: From a NumPy array
import numpy as np
data = np.array([[1, 4], [2, 5], [3, 6]])
columns = ['col1', 'col2']
df_from_array = pd.DataFrame(data, columns=columns)
print("\nDataFrame from NumPy array:\n", df_from_array)

# Method 4: From another DataFrame (copying or selecting specific columns/rows)
df_copy = df.copy() #creates a copy of the original dataframe named df
print("\nDataFrame copy:\n", df_copy.head()) #print only the first few rows for large datasets
df_subset = df[['Type 1', 'HP']] # Select specific columns into new dataframe
print("\nDataFrame subset:\n", df_subset.head())

# Method 5:  Empty DataFrame with specified column names
df_empty = pd.DataFrame(columns=['A', 'B', 'C'])

# Print the created DataFrames (optional)
print("\nEmpty DataFrame:\n", df_empty)

## Concatenating DataFrames

In [ ]:
# prompt: example of the concat method usage

import pandas as pd

# Sample DataFrames (replace with your actual DataFrames)
data1 = {'col1': [1, 2, 3], 'col2': [4, 5, 6]}
df1 = pd.DataFrame(data1)

data2 = {'col1': [7, 8, 9], 'col2': [10, 11, 12]}
df2 = pd.DataFrame(data2)

# Concatenate DataFrames along rows (axis=0, default)
df_concat_rows = pd.concat([df1, df2])
print("Concatenated along rows:\n", df_concat_rows)
print("Both rows with index 0 are selected with df_concat_rows.loc[0]:\n", df_concat_rows.loc[0])

# Ignoring index
df_ignore_index = pd.concat([df1, df2], ignore_index=True)
print("\nConcatenated along rows with ignored index:\n", df_ignore_index)

# Concatenate DataFrames along columns (axis=1)
df_concat_cols = pd.concat([df1, df2], axis=1)
print("\nConcatenated along columns:\n", df_concat_cols)

# Concatenate with keys to create a hierarchical index
df_concat_keys = pd.concat([df1, df2], keys=['df1', 'df2'])
print("\nConcatenated with keys:\n", df_concat_keys)

# Accessing data with hierarchical index
print("\nAccessing data from hierarchical index:")
print(df_concat_keys.loc['df1']) # Accessing data in df1

# Handling different column names (outer join)
data3 = {'col3': [13, 14, 15], 'col4': [16, 17, 18]}
df3 = pd.DataFrame(data3)

df_concat_outer = pd.concat([df1, df3], axis=1)
print("\nConcatenated with different column names (outer join):\n", df_concat_outer)

# Handling different column names (inner join)
df_concat_inner = pd.concat([df1, df3], axis=1, join="inner") # Only common columns will be kept
print("\nConcatenated with different column names (inner join):\n", df_concat_inner) # Should show nothing as no common columns


## Saving our Data (Exporting into Desired Format)

In [ ]:
# df.to_csv('modified.csv', index=False)

#df.to_excel('modified.xlsx', index=False)

df.to_csv('modified.txt', index=False, sep='\t')




## Plotting

In [ ]:
# prompt: Draw the histogram of the pokemons' HP values. Use dataframe.plot method for this

import matplotlib.pyplot as plt

df['HP'].plot(kind='hist', bins=20)  # Adjust the number of bins as needed
plt.xlabel('HP Value')
plt.ylabel('Frequency')
plt.title('Histogram of Pokemon HP Values')
plt.show()

In [ ]:
df[df['Type 1'] == 'Grass']  .plot(kind='scatter', x='Attack', y='Defense', color='red',  label='Grass')
plt.xlabel('Attack')
plt.ylabel('Defense')
plt.title('Attack vs. Defense')
plt.show()

In [ ]:
# prompt: Draw scatter plot of the Attack vs. Defense values. Use built-in pandas methods for this. Plot it for Grass and Psychic type 1 pokemons to one single figure. Use pandas methods for this!

# Filter for Grass and Psychic type 1 Pokémon
grass_pokemon = df[df['Type 1'] == 'Grass']
psychic_pokemon = df[df['Type 1'] == 'Psychic']

plt.scatter(grass_pokemon['Attack'], grass_pokemon['Defense'], label='Grass', alpha=0.7, color='green')
plt.scatter(psychic_pokemon['Attack'], psychic_pokemon['Defense'], label='Psychic', alpha=0.7, color='violet')

plt.xlabel('Attack')
plt.ylabel('Defense')
plt.legend()
plt.show()

In [ ]:
# prompt: Plot bar chart comparing the HP value of Grass and Fire type 1 pokemons. Make 20 bins in HP and normalize the histograms before plotting


plt.hist([grass_pokemon['HP'], psychic_pokemon['HP']], bins=10, label=['Grass', 'Psychic'], color=['green', 'violet'], density=True)
plt.xlabel('HP Value')
plt.ylabel('Normalized frequency')
plt.legend()
plt.show()

In [ ]:
# prompt: draw frequencies of the different pokemon type 1 values

# Count the frequencies of each Type 1
type1_counts = df['Type 1'].value_counts()

# Create the bar plot
plt.figure(figsize=(10, 10))  # Adjust figure size for better readability
type1_counts.plot(kind='pie')
plt.show()

In [ ]:

# Count the frequencies of each Type 1
type1_counts = df['Type 1'].value_counts()

# Create the bar plot
type1_counts.plot(kind='bar')
plt.xlabel("Pokemon Type 1")
plt.ylabel("Frequency")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show()